<a href="https://colab.research.google.com/github/LaputMarkDanielle/CPE-201L-DSA-2-A/blob/main/PROGRESS%20REPORT%203.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import threading
import time
import sqlite3
from datetime import datetime, timedelta
from tkinter import Tk, Toplevel, Label, Button, StringVar, OptionMenu, Frame, Scrollbar, Canvas, RIGHT, LEFT, BOTH, Y
from tkcalendar import Calendar
from playsound import playsound
from tkinter import filedialog

# --- CONFIG ---
DB_FILE = "tasks.db"
SOUNDS_FOLDER = "sounds"
SNOOZE_MINUTES = 5

# Ensure sounds folder exists
if not os.path.exists(SOUNDS_FOLDER):
    os.makedirs(SOUNDS_FOLDER)

DEFAULT_SOUND = os.path.join(SOUNDS_FOLDER, "ding.wav")
if not os.path.exists(DEFAULT_SOUND):
    open(DEFAULT_SOUND, 'a').close()  # placeholder ding

# --- DATABASE SETUP ---
conn = sqlite3.connect(DB_FILE, check_same_thread=False)
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS tasks (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    due_datetime TEXT,
    priority TEXT,
    alarm_type TEXT,
    notes TEXT,
    custom_sound TEXT,
    status TEXT
)
""")
conn.commit()


# --- FUNCTIONS ---

# Play sound safely in a thread
def play_sound(file_path):
    if os.path.exists(file_path):
        threading.Thread(target=playsound, args=(file_path,), daemon=True).start()


# Fetch active tasks
def get_active_tasks():
    cursor.execute("SELECT * FROM tasks WHERE status!='done' ORDER BY due_datetime")
    return cursor.fetchall()


# Fetch history tasks
def get_history_tasks():
    cursor.execute("SELECT * FROM tasks WHERE status='done' ORDER BY due_datetime DESC")
    return cursor.fetchall()


# Refresh main UI
def refresh_ui():
    build_task_list()
    build_history_list()


# --- Task Reminder Popup ---
def show_task_popup(task):
    task_id, name, due_datetime, priority, alarm_type, notes, custom_sound, status = task
    popup = Toplevel()
    popup.title("Task Reminder")
    popup.geometry("350x180")
    Label(popup, text=f"Task: {name}", font=("Arial", 12, "bold")).pack(pady=10)
    Label(popup, text=f"Due: {due_datetime}", font=("Arial", 10)).pack(pady=5)
    Label(popup, text=f"Priority: {priority}", font=("Arial", 10)).pack(pady=5)

    def mark_done():
        cursor.execute("UPDATE tasks SET status='done' WHERE id=?", (task_id,))
        conn.commit()
        refresh_ui()
        popup.destroy()

    def snooze():
        due_dt = datetime.strptime(due_datetime, "%Y-%m-%d %H:%M")
        new_due_dt = due_dt + timedelta(minutes=SNOOZE_MINUTES)
        cursor.execute("UPDATE tasks SET due_datetime=? WHERE id=?", (new_due_dt.strftime("%Y-%m-%d %H:%M"), task_id))
        conn.commit()
        refresh_ui()
        popup.destroy()

    Button(popup, text="Mark Done", width=12, command=mark_done).pack(side=LEFT, padx=20, pady=20)
    Button(popup, text=f"Snooze {SNOOZE_MINUTES} min", width=15, command=snooze).pack(side=RIGHT, padx=20, pady=20)

    # Play sound
    if custom_sound and os.path.exists(custom_sound):
        play_sound(custom_sound)
    else:
        play_sound(DEFAULT_SOUND)


# --- Alarm Scheduler ---
def alarm_checker():
    triggered = set()
    while True:
        now = datetime.now()
        tasks = get_active_tasks()
        for task in tasks:
            task_id, name, due_dt_str, priority, alarm_type, notes, custom_sound, status = task
            due_dt = datetime.strptime(due_dt_str, "%Y-%m-%d %H:%M")
            if due_dt <= now and task_id not in triggered:
                triggered.add(task_id)
                threading.Thread(target=show_task_popup, args=(task,), daemon=True).start()
        time.sleep(10)


# --- Add/Edit Task Popup ---
def add_edit_task_popup(task=None):
    popup = Toplevel()
    popup.title("Add/Edit Task")
    popup.geometry("400x350")

    # Task Name
    Label(popup, text="Task Name:").grid(row=0, column=0, sticky='w', pady=5)
    task_name_var = StringVar(value=task[1] if task else "")
    task_name_entry = Label(popup, textvariable=task_name_var)
    task_name_entry.grid(row=0, column=1, pady=5)

    # Date picker
    Label(popup, text="Date:").grid(row=1, column=0, sticky='w')
    cal = Calendar(popup, selectmode='day')
    if task:
        due_dt = datetime.strptime(task[2], "%Y-%m-%d %H:%M")
        cal.selection_set(due_dt.date())
    cal.grid(row=1, column=1, pady=5)

    # Time picker
    Label(popup, text="Time:").grid(row=2, column=0, sticky='w')
    hour_var = StringVar(value="12")
    hour_menu = OptionMenu(popup, hour_var, *[f"{i:02}" for i in range(1, 13)])
    hour_menu.grid(row=2, column=1, sticky='w')
    minute_var = StringVar(value="00")
    minute_menu = OptionMenu(popup, minute_var, *[f"{i:02}" for i in range(0, 60)])
    minute_menu.grid(row=2, column=1, sticky='e')
    ampm_var = StringVar(value="AM")
    ampm_menu = OptionMenu(popup, ampm_var, "AM", "PM")
    ampm_menu.grid(row=2, column=2, sticky='w')

    # Priority
    Label(popup, text="Priority:").grid(row=3, column=0, sticky='w')
    priority_var = StringVar(value=task[3] if task else "Medium")
    priority_menu = OptionMenu(popup, priority_var, "High", "Medium", "Low")
    priority_menu.grid(row=3, column=1, sticky='w')

    # Alarm Type
    Label(popup, text="Alarm Type:").grid(row=4, column=0, sticky='w')
    alarm_var = StringVar(value=task[4] if task else "Notification+Sound")
    alarm_menu = OptionMenu(popup, alarm_var, "Ring+Vibrate", "Vibrate only", "Notification+Sound")
    alarm_menu.grid(row=4, column=1, sticky='w')

    # Notes
    Label(popup, text="Notes:").grid(row=5, column=0, sticky='w')
    notes_var = StringVar(value=task[5] if task else "")
    notes_entry = Label(popup, textvariable=notes_var)
    notes_entry.grid(row=5, column=1, pady=5)

    # Custom Sound
    Label(popup, text="Custom Sound:").grid(row=6, column=0, sticky='w')
    custom_sound_var = StringVar(value=task[6] if task else "")

    def upload_sound():
        file_path = filedialog.askopenfilename(filetypes=[("Audio Files", "*.mp3 *.wav")])
        if file_path:
            custom_sound_var.set(file_path)

    Button(popup, text="Upload", command=upload_sound).grid(row=6, column=1, sticky='w')

    def preview_sound():
        if custom_sound_var.get():
            play_sound(custom_sound_var.get())

    Button(popup, text="Preview", command=preview_sound).grid(row=6, column=2, sticky='w')

    def reset_sound():
        custom_sound_var.set("")

    Button(popup, text="Reset", command=reset_sound).grid(row=6, column=3, sticky='w')

    # Save
    def save_task():
        date_val = cal.selection_get().strftime("%Y-%m-%d")
        hour = int(hour_var.get())
        minute = int(minute_var.get())
        if ampm_var.get() == "PM" and hour != 12:
            hour += 12
        if ampm_var.get() == "AM" and hour == 12:
            hour = 0
        due_datetime = f"{date_val} {hour:02}:{minute:02}"

        name = task_name_var.get()
        priority = priority_var.get()
        alarm_type = alarm_var.get()
        notes = notes_var.get()
        custom_sound = custom_sound_var.get()

        if task:
            cursor.execute(
                """UPDATE tasks SET name=?, due_datetime=?, priority=?, alarm_type=?, notes=?, custom_sound=? WHERE id=?""",
                (name, due_datetime, priority, alarm_type, notes, custom_sound, task[0]))
        else:
            cursor.execute("""INSERT INTO tasks (name, due_datetime, priority, alarm_type, notes, custom_sound, status)
                              VALUES (?, ?, ?, ?, ?, ?, 'active')""",
                           (name, due_datetime, priority, alarm_type, notes, custom_sound))
        conn.commit()
        refresh_ui()
        popup.destroy()

    Button(popup, text="Save Task", width=20, command=save_task).grid(row=7, column=0, columnspan=4, pady=10)


# --- Build Active Task List ---
def build_task_list():
    for widget in task_frame.winfo_children():
        widget.destroy()
    tasks = get_active_tasks()
    for t in tasks:
        t_id, name, due_dt, priority, alarm_type, notes, custom_sound, status = t
        f = Frame(task_frame, bd=1, relief='raised', padx=5, pady=5)
        Label(f, text=f"{name} ({priority})\nDue: {due_dt}", justify='left').pack(side=LEFT)
        Button(f, text="Edit", command=lambda task=t: add_edit_task_popup(task)).pack(side=RIGHT, padx=5)
        f.pack(fill='x', pady=2)


# --- Build History List ---
def build_history_list():
    for widget in history_frame.winfo_children():
        widget.destroy()
    tasks = get_history_tasks()
    for t in tasks:
        t_id, name, due_dt, priority, alarm_type, notes, custom_sound, status = t
        f = Frame(history_frame, bd=1, relief='sunken', padx=5, pady=5)
        Label(f, text=f"{name} ({priority})\nDue: {due_dt}", justify='left').pack(side=LEFT)
        Button(f, text="Do Again", command=lambda task=t: add_edit_task_popup(task)).pack(side=RIGHT, padx=5)
        f.pack(fill='x', pady=2)


# --- MAIN APP ---
app = Tk()
app.title("Smart Task Manager")
app.geometry("600x600")

# Frames for active tasks and history
Label(app, text="Active Tasks", font=("Arial", 14, "bold")).pack()
task_frame = Frame(app)
task_frame.pack(fill=BOTH, expand=True, padx=5, pady=5)

Label(app, text="History", font=("Arial", 14, "bold")).pack()
history_frame = Frame(app)
history_frame.pack(fill=BOTH, expand=True, padx=5, pady=5)

Button(app, text="Add Task", width=20, command=lambda: add_edit_task_popup()).pack(pady=10)

refresh_ui()

# Start alarm checker thread
threading.Thread(target=alarm_checker, daemon=True).start()

app.mainloop()